# Training The Model

In [2]:
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from scipy.io import wavfile

No Problems


In [ ]:
# Set file paths and parameters
win_size = 1024
hop_size = win_size // 2

In [ ]:
# Initialize arrays to hold training data
stft_mag_train = []
stft_noise_mag_train = []
stft_mag_target_train = []

In [ ]:
def trim_wav( originalWavPath,Name):
    sampleRate, waveData = wavfile.read( originalWavPath )
    startSample = int( 0 * sampleRate )
    endSample = int( 10 * sampleRate )
    wavfile.write( Name, sampleRate, waveData[startSample:endSample])

In [ ]:
# Creating the Datasets
# Loop over audio files to add noise and extract features
for FileNum in range (10):
    for NoiseNum in range (10):
        # Load the Audio File
        audio_file = ''
        trim_wav(audio_file,'audioFile.wav')
        noise_file = ''
        trim_wav(noise_file,'noiseFile.wav')
        audio, sr = librosa.load('audioFile.wav', sr=None)
        noise, sr = librosa.load('noiseFile.wav', sr=None)

        # Add noise to audio
        noisy_audio = audio + noise

        # Extract features using Short-Time Fourier Transform (STFT)
        stft_audio = librosa.stft(noisy_audio, n_fft=win_size, hop_length=hop_size)
        stft_noise = librosa.stft(noise, n_fft=win_size, hop_length=hop_size)

        # Separate magnitude and phase components
        stft_mag = np.abs(stft_audio)
        stft_phase = np.angle(stft_audio)
        stft_noise_mag = np.abs(stft_noise)

        # Reshape magnitude component to (n_frames, n_bins) format
        mag_shape = stft_mag.shape
        stft_mag_reshaped = np.reshape(stft_mag, (mag_shape[0], mag_shape[1], 1))
        stft_noise_mag_reshaped = np.reshape(stft_noise_mag, (mag_shape[0], mag_shape[1], 1))

        # Append processed features and target values to arrays
        stft_mag_train.append(stft_mag_reshaped)
        stft_noise_mag_train.append(stft_noise_mag_reshaped)
        stft_mag_target_train.append(stft_mag_reshaped)

In [ ]:
# Concatenate training data from all files
stft_mag_train = np.concatenate(stft_mag_train, axis=0)
stft_noise_mag_train = np.concatenate(stft_noise_mag_train, axis=0)
stft_mag_target_train = np.concatenate(stft_mag_target_train, axis=0)

In [ ]:
# Define RNN model with LSTM layers
model = Sequential()
model.add(LSTM(units=128, input_shape=(None, 1), return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(TimeDistributed(Dense(units=1, activation='linear')))

# Compile model
model.compile(loss='mse', optimizer='adam')

# Train model on noisy audio and noise
model.fit(np.concatenate((stft_mag_train, stft_noise_mag_train), axis=-1), stft_mag_target_train, epochs=50)

# Testing The AI Model

In [ ]:
Test_file = ''
trim_wav(Test_file,'TestFile.wav')
noise_file = ''
trim_wav(noise_file,'noiseFile.wav')
Test, sr = librosa.load('TestFile.wav', sr=None)
noise, sr = librosa.load('noiseFile.wav', sr=None)

In [ ]:
# Extract features using STFT
stft_test_audio = librosa.stft(Test, n_fft=win_size, hop_length=hop_size)
stft_noise_audio = librosa.stft(noise, n_fft=win_size, hop_length=hop_size)

In [ ]:
# Separate magnitude and phase components
stft_test_mag = np.abs(stft_test_audio)
stft_test_phase = np.angle(stft_test_audio)
stft_noise_mag = np.abs(stft_noise_audio)

# Reshape magnitude component to (n_frames, n_bins) format
test_mag_shape = stft_test_mag.shape
stft_test_mag = np.reshape(stft_test_mag, (test_mag_shape[0], test_mag_shape[1], 1))
noise_mag_shape = stft_noise_mag.shape
stft_noise_mag = np.reshape(stft_noise_mag, (noise_mag_shape[0], noise_mag_shape[1], 1))

In [ ]:
# Apply model to test audio
stft_test_mag_pred = model.predict(np.concatenate((stft_test_mag, stft_noise_mag), axis=-1))
stft_test_mag_pred = np.squeeze(stft_test_mag_pred, axis=-1)

# Invert STFT to obtain noise-reduced audio
stft_test_pred = stft_test_mag_pred * np.exp(1j * stft_test_phase)
test_audio_pred = librosa.istft(stft_test_pred, hop_length=hop_size)

In [ ]:
# Save noise-reduced audio to file
librosa.output.write_wav('ANC.wav', test_audio_pred, sr)